In [60]:
# https://medium.com/@datatec.studio
# https://huggingface.co/learn/nlp-course

!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate -U

In [61]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [63]:
# Check what is inside row_dataset
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [62]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [64]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [65]:
#  Check there is no mistake in the data processing
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 73]),
 'token_type_ids': torch.Size([8, 73]),
 'attention_mask': torch.Size([8, 73])}

In [66]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(1.1856, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [68]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [69]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

1377


In [70]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [71]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [72]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8357843137254902, 'f1': 0.8854700854700854}

In [73]:
# Mount google driver
# Please create a HF_NPL_Course Folder in your google driver at first
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/HF_NPL_Course

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HF_NPL_Course


In [77]:
# Save tokenizer and model to google driver
trained_model_dir = '/content/drive/MyDrive/HF_NPL_Course/model/bert-base-uncased_mrpc'
tokenizer.save_pretrained(trained_model_dir)
model.save_pretrained(trained_model_dir)

In [78]:
# Load tokenizer and model from google driver
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(trained_model_dir)
model = AutoModelForSequenceClassification.from_pretrained(trained_model_dir)

In [80]:
# Loop to continuously prompt the user for input and print the predicted label
while True:
    # Prompt the user for input
    sequence_1 = input("Enter the first sentence: ")
    sequence_2 = input("Enter the second sentence: ")

    # Encode the input sequences using the tokenizer
    model_inputs = tokenizer(sequence_1, sequence_2, padding=True, truncation=True, return_tensors="pt")

    # Make a prediction using the loaded model
    output = model(**model_inputs)

    # Get the predicted label from the output
    predicted_label = torch.argmax(output.logits, dim=1).item()

    # Print the predicted label
    if predicted_label == 1:
        print("The two sentences have the same meaning.")
    else:
        print("The two sentences do not have the same meaning.")

Enter the first sentence: Tomorrow is another day
Enter the second sentence: Have a nice day!
The two sentences do not have the same meaning.
Enter the first sentence: The cat is on a sofa
Enter the second sentence: Cat is lounging on the sofa
The two sentences have the same meaning.


KeyboardInterrupt: ignored